Created By: Anshoo Mehra
Dated: September 21st 2017

# Tensorflow Guide Chronological Order ..

All Credit to Udacity, all labs & explanation below is excerpt from class labs. Intent is to try all key examples & keep refrence to key highlights for future reference.

I created this notebook for personal reference, however if this helps anyone in general please feel free to refer & consider Udacity policy/lceinses prior sharing ahead. I apologize documentation is fairly limited, it is just to the point to revise concepts than using this as primary guide to learn .. 

### Hello World!

In [1]:
import tensorflow as tf

# Create TensorFlow object called hello_constant
hello_constant = tf.constant('Hello World!')

with tf.Session() as sess:
    # Run the tf.constant operation in the session
    output = sess.run(hello_constant)
    print(output)

b'Hello World!'


### Placeholders (Can't be modified like as Constant shown above)

In [2]:
import tensorflow as tf

def run():
    output = None
    x = tf.placeholder(tf.int32)

    with tf.Session() as sess:
        # TODO: Feed the x tensor 123
        output = sess.run(x, feed_dict={x:123})

    return output

print(run())

123


### Variable

In [3]:
vx = tf.Variable(5)

# The tf.Variable class creates a tensor with an initial value that
# can be modified, much like a normal Python variable. This tensor
# stores its state in the session, so you must initialize the state
# of the tensor manually. You'll use the tf.global_variables_initializer()
# function to initialize the state of all the Variable tensors.

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    print(sess.run(vx))
    print(sess.run(vx.assign(123)))
    print(sess.run(vx.assign_add(1)))

5
123
124


### Math Functions

In [4]:
x = tf.add(5, 2) #7
y = tf.subtract(10, 4) # 6
z = tf.multiply(2, 5)  # 10
f = tf.div(4,2)

with tf.Session() as sess:
        x_ = sess.run(x)
        y_ = sess.run(y)
        z_ = sess.run(z)
        f_ = sess.run(f)

print (x_, y_, z_, f_)

7 6 10 2


### Converting Types

In [78]:
#with tf.Session() as sess:
    #sess.run(tf.subtract(tf.constant(2.0),tf.constant(1))) 

## TRY TO UNCOMMENT ABOVE TO SEE THE ERROR MESSAGE..

# Fails with ValueError: Tensor conversion requested dtype float32 for Tensor with dtype int32:
# That's because the constant 1 is an integer but the constant 2.0 
# is a floating point value and subtract expects them to match.# In cases like these, you can either make sure your data is all 
# of the same type, or you can cast a value to another type. In this case,
# converting the 2.0 to an integer before subtracting, like so, will give 
# the correct result:

In [6]:
# In cases like these, you can either make sure your data is all 
# of the same type, or you can cast a value to another type. In this case,
# converting the 2.0 to an integer before subtracting, like so, will give 
# the correct result:

dummy = tf.subtract(tf.cast(tf.constant(2.0), tf.int32), tf.constant(1))   # 1

with tf.Session() as sess:
    print (sess.run(dummy))

1


### Example Math Function & Type Conversion

In [7]:
import tensorflow as tf

# TODO: Convert the following to TensorFlow:
#x = 10
#y = 2
#z = x/y - 1

# TODO: Print z from a session
x = tf.constant(10)
y = tf.constant(2)
z = tf.subtract(tf.div(x,y),tf.constant(1))
f = tf.cast(z, tf.float32)
## OR z = tf.div(x,y) - 1

with tf.Session() as sess:
    output = sess.run(z)
    print(output)
    output = sess.run(f)
    print(output)
    

4
4.0


### Assuming everyone knows Linear Combination Equation y = xW + b 

y = Logits
x = Inputs
W = Weights
b = Bias

Linear Equation will give is Logits

### Python Softmax

Softmax Function Convert Logits to Probabilities (All Probabilities Add to 1) 

In [18]:
import numpy as np


def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    # TODO: Compute and return softmax(x)
    return np.exp(x)/np.sum(np.exp(x), axis=0)
        
logits = [3.0, 1.0, 0.2]
print(softmax(logits))

[ 0.8360188   0.11314284  0.05083836]


### Softmax in Tensorflow

In [21]:
import tensorflow as tf


def run():
    output = None
    logit_data = [2.0, 1.0, 0.1]
    logits = tf.placeholder(tf.float32)

    softmax = tf.nn.softmax(logits)

    with tf.Session() as sess:
        output = sess.run(softmax, feed_dict={logits: logit_data})

    return output

print(run())

[ 0.65900117  0.24243298  0.09856589]


### One-Hot Encoding 

y = [ .5, .6, .9]

y_one_hot_encoded = [ 0, 0, 1]

So the hightest probability gets marked 1 and rest 0 as simple as that ...

One Hot Encoding works really well but One problem with One Hot Encoding is that once classes grow to thousands or millions, vector becomes really large & very inefficient. This can be solved with embeddings, we will see that later.

However, this method makes the process very simple, we have just 2 vectors, one vector contains probabilities of classes and other vector one-hot encodeding for labels. So if we have a way to measure distance between these two Vectors, that can help us classify and this process of measuring distance is called **Cross-Entropy.

**Cross-Entropy will give us small distance for Right Class and higher distance for Incorrect Classes.**

### Cross Entropy

D(S,L) = - Σᵢ Lᵢ * log(Sᵢ)



### Overall Process is called " Multinomial Logistics Classification "


INPUT **--linear model--**> LOGITS > **--softmax--** > PROBABILITIES > **--cross_entropy--** < 1_HOT_LABELS





#### Let's see how to compute Cross Entropy in Tesnforflow 

In [22]:
import tensorflow as tf

softmax_data = [0.7, 0.2, 0.1]
one_hot_data = [1.0, 0.0, 0.0]

softmax = tf.placeholder(tf.float32)
one_hot = tf.placeholder(tf.float32)

# ToDo: Print cross entropy from session
cross_entropy = -tf.reduce_sum(tf.multiply(one_hot, tf.log(softmax)))

with tf.Session() as sess:
    print(sess.run(cross_entropy, feed_dict={softmax: softmax_data, one_hot: one_hot_data}))


0.356675


### Almost there, since most of us will be trying to train models on CPU, concept called mini-batching is something we must learn to not run short on memory and be stranded in middle of nowhere ..

Mini-batching is a technique for training on subsets of the dataset instead of all the data at one time. This provides the ability to train a model, even if a computer lacks the memory to store the entire dataset.

Mini-batching is computationally inefficient, since you can't calculate the loss simultaneously across all samples. However, this is a small price to pay in order to be able to run the model at all.

It's also quite useful combined with SGD. The idea is to randomly shuffle the data at the start of each epoch, then create the mini-batches. For each mini-batch, you train the network weights with gradient descent. Since these batches are random, you're performing SGD with each batch.

**TensorFlow Mini-batching**
In order to use mini-batching, you must first divide your data into batches.

Unfortunately, it's sometimes impossible to divide the data into batches of exactly equal size. For example, imagine you'd like to create batches of 128 samples each from a dataset of 1000 samples. Since 128 does not evenly divide into 1000, you'd wind up with 7 batches of 128 samples, and 1 batch of 104 samples. (7*128 + 1*104 = 1000)

In that case, the size of the batches would vary, so you need to take advantage of TensorFlow's tf.placeholder() function to receive the varying batch sizes.

Continuing the example, if each sample had n_input = 784 features and n_classes = 10 possible labels, the dimensions for features would be [None, n_input] and labels would be [None, n_classes].

features = tf.placeholder(tf.float32, [**None**, n_input]) <br>
labels = tf.placeholder(tf.float32, [**None**, n_classes])

What does None do here?

The None dimension is a placeholder for the batch size. At runtime, TensorFlow will accept any batch size greater than 0.

Going back to our earlier example, this setup allows you to feed features and labels into the model as either the batches of 128 samples or the single batch of 104 samples.

Let's understand it better with a example .. 

In [53]:
import math
from pprint import pprint


def batches(batch_size, features, labels):
    """
    Create batches of features and labels
    :param batch_size: The batch size
    :param features: List of features
    :param labels: List of labels
    :return: Batches of (Features, Labels)
    """
    assert len(features) == len(labels)
    # TODO: Implement batching
    outout_batches = []
    
    sample_size = len(features)
    for start_i in range(0, sample_size, batch_size):
        end_i = start_i + batch_size
        batch = [features[start_i:end_i], labels[start_i:end_i]]
        outout_batches.append(batch)
        
    return outout_batches


# 4 Samples of features
example_features = [
    ['F11','F12','F13','F14'],
    ['F21','F22','F23','F24'],
    ['F31','F32','F33','F34'],
    ['F41','F42','F43','F44']]

# 4 Samples of labels
example_labels = [
    ['L11','L12'],
    ['L21','L22'],
    ['L31','L32'],
    ['L41','L42']]

example_batches = batches(3, example_features, example_labels)

pprint(example_batches)

[[[['F11', 'F12', 'F13', 'F14'],
   ['F21', 'F22', 'F23', 'F24'],
   ['F31', 'F32', 'F33', 'F34']],
  [['L11', 'L12'], ['L21', 'L22'], ['L31', 'L32']]],
 [[['F41', 'F42', 'F43', 'F44']], [['L41', 'L42']]]]


### Epochs 

With mini-batching the accuracy is low, but you probably know that you could train on the dataset more than once. You can train a model using the dataset multiple times.

An **epoch** is a single forward and backward pass of the whole dataset. This is used to increase the accuracy of the model without requiring more data. This section will cover epochs in TensorFlow and how to choose the right number of epochs.

## So, let's put it all together and run the below lab. 

**If it appear confusing, try to going to last commneted lab & understand one step at a time & if need be going back to above examples .. 

### Handwritten Numbers (Image) Classification using MNIST database by applying ..
### Linear Function / Logistics Regression ( y= xW + b )with Tensforflow


Initializing the weights with random numbers from a normal distribution is good practice. Randomizing the weights helps the model from becoming stuck in the same place every time you train it.

Similarly, choosing weights from a normal distribution prevents any one weight from overwhelming other weights. You'll use the tf.truncated_normal() function to generate random numbers from a normal distribution.

The tf.truncated_normal() function returns a tensor with random values from a normal distribution whose magnitude is no more than 2 standard deviations from the mean.

Since the weights are already helping prevent the model from getting stuck, you don't need to randomize the bias.

In [67]:
import tensorflow as tf

def weights(n_features, n_labels):
    """
    Return TensorFlow weights
    :param n_features: Number of features
    :param n_labels: Number of labels
    :return: TensorFlow weights
    """
    # TODO: Return weights
    return tf.Variable(tf.random_normal([n_features, n_labels]))


def biases(n_labels):
    """
    Return TensorFlow bias
    :param n_labels: Number of labels
    :return: TensorFlow bias
    """
    # TODO: Return biases
    return tf.Variable(tf.random_normal[n_labels])


def linear(input, w, b):
    """
    Return linear function in TensorFlow
    :param input: TensorFlow input
    :param w: TensorFlow weights
    :param b: TensorFlow biases
    :return: TensorFlow linear function
    """
    # TODO: Linear Function (xW + b)
    return tf.add(tf.matmul(input, w), b)

def print_epoch_stats(epoch_i, sess, last_features, last_labels):
    """
    Print cost and validation accuracy of an epoch
    """
    current_cost = sess.run(
        cost,
        feed_dict={features: last_features, labels: last_labels})
    valid_accuracy = sess.run(
        accuracy,
        feed_dict={features: valid_features, labels: valid_labels})
    print('Epoch: {:<4} - Cost: {:<8.3} Valid Accuracy: {:<5.3}'.format(
        epoch_i,
        current_cost,
        valid_accuracy))
    
def batches(batch_size, features, labels):
    """
    Create batches of features and labels
    :param batch_size: The batch size
    :param features: List of features
    :param labels: List of labels
    :return: Batches of (Features, Labels)
    """
    assert len(features) == len(labels)
    # TODO: Implement batching
    outout_batches = []
    
    sample_size = len(features)
    for start_i in range(0, sample_size, batch_size):
        end_i = start_i + batch_size
        batch = [features[start_i:end_i], labels[start_i:end_i]]
        outout_batches.append(batch)
        
    return outout_batches

In [75]:
n_input = 784  # MNIST data input (img shape: 28*28)
n_classes = 10  # MNIST total classes (0-9 digits)

# Import MNIST data
mnist = input_data.read_data_sets('/tmp/tensorflow/mnist/input_data', one_hot=True)

# The features are already scaled and the data is shuffled
train_features = mnist.train.images
valid_features = mnist.validation.images
test_features = mnist.test.images

train_labels = mnist.train.labels.astype(np.float32)
valid_labels = mnist.validation.labels.astype(np.float32)
test_labels = mnist.test.labels.astype(np.float32)

# Features and Labels
features = tf.placeholder(tf.float32, [None, n_input])
labels = tf.placeholder(tf.float32, [None, n_classes])

# Weights & bias
weights = tf.Variable(tf.random_normal([n_input, n_classes]))
bias = tf.Variable(tf.random_normal([n_classes]))

# Logits - xW + b
logits = tf.add(tf.matmul(features, weights), bias)

# Define loss and optimizer
learning_rate = tf.placeholder(tf.float32)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Calculate accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init = tf.global_variables_initializer()

batch_size = 128
epochs = 500
learn_rate = 0.09

train_batches = batches(batch_size, train_features, train_labels)

with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch_i in range(epochs):

        # Loop over all batches
        for batch_features, batch_labels in train_batches:
            train_feed_dict = {
                features: batch_features,
                labels: batch_labels,
                learning_rate: learn_rate}
            sess.run(optimizer, feed_dict=train_feed_dict)

        # Print cost and validation accuracy of an epoch
        print_epoch_stats(epoch_i, sess, batch_features, batch_labels)

    # Calculate accuracy for test dataset
    test_accuracy = sess.run(
        accuracy,
        feed_dict={features: test_features, labels: test_labels})

print('Test Accuracy: {}'.format(test_accuracy))

Extracting /tmp/tensorflow/mnist/input_data/train-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/train-labels-idx1-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-labels-idx1-ubyte.gz
Epoch: 0    - Cost: 1.78     Valid Accuracy: 0.727
Epoch: 1    - Cost: 1.27     Valid Accuracy: 0.799
Epoch: 2    - Cost: 1.06     Valid Accuracy: 0.824
Epoch: 3    - Cost: 0.939    Valid Accuracy: 0.841
Epoch: 4    - Cost: 0.85     Valid Accuracy: 0.852
Epoch: 5    - Cost: 0.783    Valid Accuracy: 0.86 
Epoch: 6    - Cost: 0.731    Valid Accuracy: 0.866
Epoch: 7    - Cost: 0.69     Valid Accuracy: 0.87 
Epoch: 8    - Cost: 0.656    Valid Accuracy: 0.874
Epoch: 9    - Cost: 0.629    Valid Accuracy: 0.879
Epoch: 10   - Cost: 0.607    Valid Accuracy: 0.881
Epoch: 11   - Cost: 0.587    Valid Accuracy: 0.883
Epoch: 12   - Cost: 0.57     Valid Accuracy: 0.886
Epoch: 13   - Cost: 0.554    Valid Accuracy: 0.889
Epoc

Epoch: 156  - Cost: 0.24     Valid Accuracy: 0.921
Epoch: 157  - Cost: 0.239    Valid Accuracy: 0.922
Epoch: 158  - Cost: 0.239    Valid Accuracy: 0.921
Epoch: 159  - Cost: 0.239    Valid Accuracy: 0.921
Epoch: 160  - Cost: 0.238    Valid Accuracy: 0.921
Epoch: 161  - Cost: 0.238    Valid Accuracy: 0.921
Epoch: 162  - Cost: 0.238    Valid Accuracy: 0.922
Epoch: 163  - Cost: 0.237    Valid Accuracy: 0.922
Epoch: 164  - Cost: 0.237    Valid Accuracy: 0.922
Epoch: 165  - Cost: 0.237    Valid Accuracy: 0.921
Epoch: 166  - Cost: 0.236    Valid Accuracy: 0.921
Epoch: 167  - Cost: 0.236    Valid Accuracy: 0.922
Epoch: 168  - Cost: 0.236    Valid Accuracy: 0.922
Epoch: 169  - Cost: 0.235    Valid Accuracy: 0.922
Epoch: 170  - Cost: 0.235    Valid Accuracy: 0.922
Epoch: 171  - Cost: 0.235    Valid Accuracy: 0.922
Epoch: 172  - Cost: 0.234    Valid Accuracy: 0.922
Epoch: 173  - Cost: 0.234    Valid Accuracy: 0.922
Epoch: 174  - Cost: 0.234    Valid Accuracy: 0.922
Epoch: 175  - Cost: 0.233    Va

Epoch: 317  - Cost: 0.206    Valid Accuracy: 0.924
Epoch: 318  - Cost: 0.206    Valid Accuracy: 0.924
Epoch: 319  - Cost: 0.206    Valid Accuracy: 0.924
Epoch: 320  - Cost: 0.206    Valid Accuracy: 0.924
Epoch: 321  - Cost: 0.206    Valid Accuracy: 0.924
Epoch: 322  - Cost: 0.206    Valid Accuracy: 0.924
Epoch: 323  - Cost: 0.206    Valid Accuracy: 0.924
Epoch: 324  - Cost: 0.206    Valid Accuracy: 0.924
Epoch: 325  - Cost: 0.205    Valid Accuracy: 0.924
Epoch: 326  - Cost: 0.205    Valid Accuracy: 0.924
Epoch: 327  - Cost: 0.205    Valid Accuracy: 0.924
Epoch: 328  - Cost: 0.205    Valid Accuracy: 0.924
Epoch: 329  - Cost: 0.205    Valid Accuracy: 0.924
Epoch: 330  - Cost: 0.205    Valid Accuracy: 0.924
Epoch: 331  - Cost: 0.205    Valid Accuracy: 0.924
Epoch: 332  - Cost: 0.205    Valid Accuracy: 0.924
Epoch: 333  - Cost: 0.205    Valid Accuracy: 0.924
Epoch: 334  - Cost: 0.204    Valid Accuracy: 0.924
Epoch: 335  - Cost: 0.204    Valid Accuracy: 0.924
Epoch: 336  - Cost: 0.204    Va

Epoch: 478  - Cost: 0.193    Valid Accuracy: 0.926
Epoch: 479  - Cost: 0.192    Valid Accuracy: 0.926
Epoch: 480  - Cost: 0.192    Valid Accuracy: 0.926
Epoch: 481  - Cost: 0.192    Valid Accuracy: 0.926
Epoch: 482  - Cost: 0.192    Valid Accuracy: 0.926
Epoch: 483  - Cost: 0.192    Valid Accuracy: 0.926
Epoch: 484  - Cost: 0.192    Valid Accuracy: 0.926
Epoch: 485  - Cost: 0.192    Valid Accuracy: 0.926
Epoch: 486  - Cost: 0.192    Valid Accuracy: 0.926
Epoch: 487  - Cost: 0.192    Valid Accuracy: 0.926
Epoch: 488  - Cost: 0.192    Valid Accuracy: 0.926
Epoch: 489  - Cost: 0.192    Valid Accuracy: 0.926
Epoch: 490  - Cost: 0.192    Valid Accuracy: 0.926
Epoch: 491  - Cost: 0.192    Valid Accuracy: 0.926
Epoch: 492  - Cost: 0.192    Valid Accuracy: 0.926
Epoch: 493  - Cost: 0.192    Valid Accuracy: 0.926
Epoch: 494  - Cost: 0.192    Valid Accuracy: 0.926
Epoch: 495  - Cost: 0.191    Valid Accuracy: 0.926
Epoch: 496  - Cost: 0.191    Valid Accuracy: 0.926
Epoch: 497  - Cost: 0.191    Va

In [68]:
### SOMETHING TO TRY TO GET HANG OF IT ... IF ABOVE IS TOO COMPLEX .. 

# import tensorflow as tf
# from tensorflow.examples.tutorials.mnist import input_data
# ## Above Block Helper Functions can be modlarized to separate file .. if so
# ## below import
# #from quiz import weights, biases, linear 

# def mnist_features_labels(n_labels):
#     """
#     Gets the first <n> labels from the MNIST dataset
#     :param n_labels: Number of labels to use
#     :return: Tuple of feature list and label list
#     """
#     mnist_features = []
#     mnist_labels = []

#     #mnist = input_data.read_data_sets('/datasets/ud730/mnist', one_hot=True)
#     mnist = input_data.read_data_sets('/tmp/tensorflow/mnist/input_data', one_hot=True)

#     # In order to make quizzes run faster, we're only looking at 10000 images
#     for mnist_feature, mnist_label in zip(*mnist.train.next_batch(10000)):
        
#         #print ("Before Checking", mnist_feature, mnist_label)
        
#         # Add features and labels if it's for the first <n>th labels
#         # In this eample since n_labels is 3, ie. features & labels for only numbers 0, 1, 2 will be retrieved
#         # from dataset remember each feature is 28*28 image and is flattened to 784 (1*784 Matrix)
#         if mnist_label[:n_labels].any():
#             #If Label is any of top 3, add feature for current index record
#             mnist_features.append(mnist_feature)
#             #If Label is any of top 3, add label values for just top 3 indexes, we do not need rest as we are not
#             # looking to classify anything after number 3..
#             mnist_labels.append(mnist_label[:n_labels])

#     return mnist_features, mnist_labels


# # Number of features (28*28 image is 784 features)
# n_features = 784
# # Number of labels
# n_labels = 3

# # Features and Labels
# features = tf.placeholder(tf.float32)
# labels = tf.placeholder(tf.float32)

# # Weights and Biases
# w = weights(n_features, n_labels)
# b = biases(n_labels)

# # Linear Function xW + b
# logits = linear(features, w, b)


# # Training data
# train_features, train_labels = mnist_features_labels(n_labels)

# with tf.Session() as session:
#     session.run(tf.global_variables_initializer())

#     # Softmax (Thus far we studied Sigmoid, it will be good to google understanding differences between them)
#     prediction = tf.nn.softmax(logits)

#     # Cross entropy
#     # This quantifies how far off the predictions were.
#     # You'll learn more about this in future lessons.
#     cross_entropy = -tf.reduce_sum(labels * tf.log(prediction), reduction_indices=1)

#     # Training loss
#     # You'll learn more about this in future lessons.
#     loss = tf.reduce_mean(cross_entropy)

#     # Rate at which the weights are changed
#     # You'll learn more about this in future lessons.
#     learning_rate = 0.08

#     # Gradient Descent
#     # This is the method used to train the model
#     # You'll learn more about this in future lessons.
#     optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

#     # Run optimizer and get loss
#     _, l = session.run(
#         [optimizer, loss],
#         feed_dict={features: train_features, labels: train_labels})

# # Print loss
# print('Loss: {}'.format(l))